Вы получили основные навыки обработки данных, теперь пора испытать их на практике. Сейчас вам предстоит заняться задачей классификации.

Представлен датасет центра приюта животных, и вашей задачей будет обучить модель таким образом, чтобы  по определенным признакам была возможность максимально уверенно предсказать метки 'Adoption' и 'Transfer' (столбец “outcome_type”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:
1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда
6. Использование профайлера вам поможет.
7. Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

In [1]:
# “SNR” stands for “shelter-neuter-return,”

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

In [3]:
data = pd.read_csv('aac_shelter_outcomes.csv')
data.head(5)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [4]:
data.shape

(78256, 12)

In [5]:
data.describe()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
count,78248,78256,78256,78256,78256,78256,78256,78256,54370,35963,78244,78254
unique,46,70855,5,2128,525,5869,64361,64361,14574,19,9,5
top,1 year,A706536,Dog,Domestic Shorthair Mix,Black/White,2014-05-05T00:00:00,2016-04-18T00:00:00,2016-04-18T00:00:00,Bella,Partner,Adoption,Neutered Male
freq,14355,11,44242,23335,8153,112,39,39,344,19660,33112,27784


In [6]:
data.animal_type.value_counts()

Dog          44242
Cat          29422
Other         4249
Bird           334
Livestock        9
Name: animal_type, dtype: int64

In [7]:
data.outcome_type.value_counts()

Adoption           33112
Transfer           23499
Return to Owner    14354
Euthanasia          6080
Died                 680
Disposal             307
Rto-Adopt            150
Missing               46
Relocate              16
Name: outcome_type, dtype: int64

In [8]:
data.outcome_subtype.value_counts()

Partner                19660
Foster                  5558
SCRP                    3211
Suffering               2514
Rabies Risk             2417
Snr                      626
Aggressive               506
Offsite                  367
In Kennel                343
Medical                  259
In Foster                182
Behavior                 142
At Vet                    59
Enroute                   45
Underage                  28
Court/Investigation       18
In Surgery                16
Possible Theft             9
Barn                       3
Name: outcome_subtype, dtype: int64

In [9]:
data.query('outcome_type == "Transfer"').outcome_subtype.value_counts()

Partner    19660
SCRP        3211
Snr          626
Barn           2
Name: outcome_subtype, dtype: int64

In [10]:
data.query('outcome_type == "Adoption"').outcome_subtype.value_counts()
# same:
# data[data.outcome_type == 'Adoption'].outcome_subtype.value_counts()

Foster     5558
Offsite     367
Barn          1
Name: outcome_subtype, dtype: int64

In [11]:
# data.query('animal_type == "Other"').head()

In [12]:
data.query('outcome_type == "Adoption"')

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
8,1 month,A685067,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,2014-08-14T18:45:00,Lucy,NaN,Adoption,Intact Female
9,3 months,A678580,Cat,Domestic Shorthair Mix,White/Black,2014-03-26T00:00:00,2014-06-29T17:45:00,2014-06-29T17:45:00,*Frida,Offsite,Adoption,Spayed Female
12,2 months,A677679,Dog,Chihuahua Shorthair/Pomeranian,Black,2014-03-07T00:00:00,2014-05-26T19:10:00,2014-05-26T19:10:00,Kash,Foster,Adoption,Neutered Male
18,2 months,A692378,Dog,German Shepherd/Labrador Retriever,Black/White,2014-10-19T00:00:00,2014-12-21T18:20:00,2014-12-21T18:20:00,*Bonnie,Foster,Adoption,Spayed Female
...,...,...,...,...,...,...,...,...,...,...,...,...
78250,1 month,A764895,Dog,Golden Retriever/Labrador Retriever,Brown/White,2017-12-04T00:00:00,2018-02-01T18:40:00,2018-02-01T18:40:00,NaN,Foster,Adoption,Neutered Male
78251,1 month,A764894,Dog,Golden Retriever/Labrador Retriever,Brown/White,2017-12-04T00:00:00,2018-02-01T18:26:00,2018-02-01T18:26:00,NaN,Foster,Adoption,Spayed Female
78252,3 years,A764468,Dog,Mastiff Mix,Blue/White,2014-12-30T00:00:00,2018-02-01T18:06:00,2018-02-01T18:06:00,Max,NaN,Adoption,Neutered Male
78254,2 months,A765858,Dog,Standard Schnauzer,Red,2017-11-13T00:00:00,2018-02-01T18:32:00,2018-02-01T18:32:00,NaN,NaN,Adoption,Spayed Female


In [13]:
# Удалим признаки, никаким образом не способные повлиять на результат.
# Все, что касается времени и возраста, позже извлечем из age_upon_outcome.

data.drop(['animal_id', 'name', 'date_of_birth', 'datetime', 'monthyear'], axis=1, inplace=True)
data.head()

,age_upon_outcome,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male
1,1 year,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female
2,1 year,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male
3,9 years,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male
4,5 months,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown


In [14]:
# Определим количество пустых значений в столбцах (по умолчанию isna().sum(axis=0), a для строк isna().sum(axis=1))

data.isna().sum()

age_upon_outcome        8
animal_type             0
breed                   0
color                   0
outcome_subtype     42293
outcome_type           12
sex_upon_outcome        2
dtype: int64

In [15]:
# Посмотрим на строки с отсутствующими значениями по столбцу outcome_type

# data.loc[data.outcome_type.isna(), :]

In [16]:
# Можно легко избавиться от строк с отсутствующими значениями по age_upon_outcome, outcome_type and sex_upon_outcome.
# Трогать outcome_subtype нельзя из-за слишком большого количества таких значений (больше половины).

data = data.loc[~data.age_upon_outcome.isna(), :]
data = data.loc[~data.outcome_type.isna(), :]
data = data.loc[~data.sex_upon_outcome .isna(), :]

In [17]:
# Взглянем на успехи по избавления от NaN

data.isna().sum()

age_upon_outcome        0
animal_type             0
breed                   0
color                   0
outcome_subtype     42280
outcome_type            0
sex_upon_outcome        0
dtype: int64

In [18]:
# Датасет совсем незначительно уменьшился, зато стал чище

data.shape

(78237, 7)

In [19]:
data.sex_upon_outcome.value_counts()

Neutered Male    27782
Spayed Female    25203
Intact Male       9544
Intact Female     9139
Unknown           6569
Name: sex_upon_outcome, dtype: int64

In [20]:
data.head(10)

,age_upon_outcome,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male
1,1 year,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female
2,1 year,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male
3,9 years,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male
4,5 months,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown
5,4 months,Dog,Leonberger Mix,Brown/White,Partner,Transfer,Intact Male
6,1 year,Other,Squirrel Mix,Tan,Suffering,Euthanasia,Unknown
7,3 years,Dog,Chihuahua Shorthair Mix,Brown,Partner,Transfer,Spayed Female
8,1 month,Cat,Domestic Shorthair Mix,Blue Tabby/White,NaN,Adoption,Intact Female
9,3 months,Cat,Domestic Shorthair Mix,White/Black,Offsite,Adoption,Spayed Female


In [21]:
# Проверим, есть ли смысл в разбиении Other на категории животных

# data.query('animal_type == "Other"').breed.value_counts().head(50)

In [22]:
# Пород животных оказалось слишком много. Не будем делать такое разбиение и загромождать датасет
# койотами и змеями, число которых чрезвычайно мало, во-первых, по сравнению с кошками и собаками,
# во-вторых, для правильного обучения модели (не на пяти же койотах ей обучаться).
# Но можно предположить, что породистость животного оказывает влияние на результат.
# Поэтому извлечем из breed отдельно породу (breed_) и отдельно породистость (breed_purity),
# разделив вторую на Mix и Pure, т.е. на смешанных и породистых.

# Создаем функцию для определения породы (если Mix есть в названии, удаляет это слово, а если нет, то переносит как есть)
def get_breed_only(x):
    x = str(x)
    mix_place = x.find('Mix') # если find ничего не находит, он выдает -1
    if mix_place == -1:
        return x
    else:
        return x[0:mix_place-1]

# Создаем функцию для определения породистости
def get_purity(x):
    x = str(x)
    if 'Mix' in x or '/' in x: # or 2 same strings: if x.count('Mix') >= 1: or if x.find('Mix') >= 0
        return 'Mix'
    else:
        return 'Pure'
    
# Создаем функцию для определения пола животного
def get_sex(x):
    if 'Male' in x:
        return 'Male'
    if 'Female' in x:
        return 'Female'
    
# Создаем функцию для определения стерилизованности животного
def get_reproduction(x):
    if 'Neutered' in x or 'Spayed' in x:
        return 'Sterilized'
    if 'Intact' in x:
        return 'Intact'

# Создаем столбцы с новыми признаками
data['breed_'] = data.breed.apply(get_breed_only)
data['breed_purity'] = data.breed.apply(get_purity)
data['sex_on_outcome'] = data.sex_upon_outcome.apply(get_sex)
data['reproduction'] = data.sex_upon_outcome.apply(get_reproduction)

# Удаляем первоначальный sex_upon_outcome и breed, поскольку они отдали нам все, что могли
data.drop(['sex_upon_outcome', 'breed'], axis=1, inplace=True)
data.head(10)

,age_upon_outcome,animal_type,color,outcome_subtype,outcome_type,breed_,breed_purity,sex_on_outcome,reproduction
0,2 weeks,Cat,Orange Tabby,Partner,Transfer,Domestic Shorthair,Mix,Male,Intact
1,1 year,Dog,White/Brown,Partner,Transfer,Beagle,Mix,Female,Sterilized
2,1 year,Dog,Blue/White,NaN,Adoption,Pit Bull,Pure,Male,Sterilized
3,9 years,Dog,White,Partner,Transfer,Miniature Schnauzer,Mix,Male,Sterilized
4,5 months,Other,Brown,Rabies Risk,Euthanasia,Bat,Mix,None,None
5,4 months,Dog,Brown/White,Partner,Transfer,Leonberger,Mix,Male,Intact
6,1 year,Other,Tan,Suffering,Euthanasia,Squirrel,Mix,None,None
7,3 years,Dog,Brown,Partner,Transfer,Chihuahua Shorthair,Mix,Female,Sterilized
8,1 month,Cat,Blue Tabby/White,NaN,Adoption,Domestic Shorthair,Mix,Female,Intact
9,3 months,Cat,White/Black,Offsite,Adoption,Domestic Shorthair,Mix,Female,Sterilized


In [23]:
# data.age_upon_outcome.value_counts()

In [24]:
# Теперь разберемся с возрастом животных.
# Чтобы не терять ценные моменты жизни до одного года, будем считать возраст в месяцах.

import re
import math

# Создаем функцию для определения возраста животных в месяцах
def get_age(x):
    if 'day' in x:
        days_num = re.search(r'\d+', x)
        return math.floor(float(days_num.group()) / 30) # можно было просто через int округлить в нижнюю сторону
    if 'week' in x:
        weeks_num = re.search(r'\d+', x)
        return math.floor(float(weeks_num.group()) / 4)
    if 'month' in x:
        months_num = re.search(r'\d+', x)
        return math.floor(float(months_num.group()))
    if 'year' in x:
        years_num = re.search(r'\d+', x)
        return math.floor(float(years_num.group()) * 12)
    
# Создаем столбец с новым признаком
data['age_on_outcome_in_months'] = data.age_upon_outcome.apply(get_age)

# Удаляем первоначальный age_upon_outcome
data.drop(['age_upon_outcome'], axis=1, inplace=True)
data.head(10)

,animal_type,color,outcome_subtype,outcome_type,breed_,breed_purity,sex_on_outcome,reproduction,age_on_outcome_in_months
0,Cat,Orange Tabby,Partner,Transfer,Domestic Shorthair,Mix,Male,Intact,0
1,Dog,White/Brown,Partner,Transfer,Beagle,Mix,Female,Sterilized,12
2,Dog,Blue/White,NaN,Adoption,Pit Bull,Pure,Male,Sterilized,12
3,Dog,White,Partner,Transfer,Miniature Schnauzer,Mix,Male,Sterilized,108
4,Other,Brown,Rabies Risk,Euthanasia,Bat,Mix,None,None,5
5,Dog,Brown/White,Partner,Transfer,Leonberger,Mix,Male,Intact,4
6,Other,Tan,Suffering,Euthanasia,Squirrel,Mix,None,None,12
7,Dog,Brown,Partner,Transfer,Chihuahua Shorthair,Mix,Female,Sterilized,36
8,Cat,Blue Tabby/White,NaN,Adoption,Domestic Shorthair,Mix,Female,Intact,1
9,Cat,White/Black,Offsite,Adoption,Domestic Shorthair,Mix,Female,Sterilized,3


In [25]:
# Создадим датасет с отдельными столбцами для каждого класса
# Adoption и Transfer выделим (по заданию требуются именно они), а все остальные объединим в Other

data['adoption'] = data['outcome_type'].apply(lambda x: 1 if x == 'Adoption' else 0)
data['transfer'] = data['outcome_type'].apply(lambda x: 1 if x == 'Transfer' else 0)
data['other_outcome_type'] = data['outcome_type'].apply(lambda x: 1 if x not in ['Adoption', 'Transfer'] else 0)

# Удалим уже ненужный Удалим уже ненужный outcome_type
data.drop(['outcome_type'], axis=1, inplace=True)

In [26]:
data

,animal_type,color,outcome_subtype,breed_,breed_purity,sex_on_outcome,reproduction,age_on_outcome_in_months,adoption,transfer,other_outcome_type
0,Cat,Orange Tabby,Partner,Domestic Shorthair,Mix,Male,Intact,0,0,1,0
1,Dog,White/Brown,Partner,Beagle,Mix,Female,Sterilized,12,0,1,0
2,Dog,Blue/White,NaN,Pit Bull,Pure,Male,Sterilized,12,1,0,0
3,Dog,White,Partner,Miniature Schnauzer,Mix,Male,Sterilized,108,0,1,0
4,Other,Brown,Rabies Risk,Bat,Mix,None,None,5,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
78250,Dog,Brown/White,Foster,Golden Retriever/Labrador Retriever,Mix,Male,Sterilized,1,1,0,0
78251,Dog,Brown/White,Foster,Golden Retriever/Labrador Retriever,Mix,Female,Sterilized,1,1,0,0
78252,Dog,Blue/White,NaN,Mastiff,Mix,Male,Sterilized,36,1,0,0
78254,Dog,Red,NaN,Standard Schnauzer,Pure,Female,Sterilized,2,1,0,0


In [27]:
# len(data.outcome_type.value_counts())

Поскольку мы будем использовать случайный лес, то нам подойдет исключительно LabelEncoder, который переведет категориальные параметры в случайные числовые (и сделает датасет нечитаемым для человека, но не для модели).

In [28]:
# Создаем функцию для замены значений, не являющихся str и int, на слово Unknown для дальнейшего использования LabelEncoder.
# Это необходимо, потому что LabelEncoder принимает только str и int.

def get_str_and_int(x):
    if type(x) == str or type(x) == int:
        return x
    else:
        return 'Unknown'

# Создаем столбцы с новыми признаками
data['outcome_subtype_'] = data.outcome_subtype.apply(get_str_and_int)
data['sex_on_outcome_'] = data.sex_on_outcome.apply(get_str_and_int)
data['reproduction_'] = data.reproduction.apply(get_str_and_int)

# Удаляем первоначальные столбцы
data.drop(['outcome_subtype', 'sex_on_outcome', 'reproduction'], axis=1, inplace=True)
data.head(10)

,animal_type,color,breed_,breed_purity,age_on_outcome_in_months,adoption,transfer,other_outcome_type,outcome_subtype_,sex_on_outcome_,reproduction_
0,Cat,Orange Tabby,Domestic Shorthair,Mix,0,0,1,0,Partner,Male,Intact
1,Dog,White/Brown,Beagle,Mix,12,0,1,0,Partner,Female,Sterilized
2,Dog,Blue/White,Pit Bull,Pure,12,1,0,0,Unknown,Male,Sterilized
3,Dog,White,Miniature Schnauzer,Mix,108,0,1,0,Partner,Male,Sterilized
4,Other,Brown,Bat,Mix,5,0,0,1,Rabies Risk,Unknown,Unknown
5,Dog,Brown/White,Leonberger,Mix,4,0,1,0,Partner,Male,Intact
6,Other,Tan,Squirrel,Mix,12,0,0,1,Suffering,Unknown,Unknown
7,Dog,Brown,Chihuahua Shorthair,Mix,36,0,1,0,Partner,Female,Sterilized
8,Cat,Blue Tabby/White,Domestic Shorthair,Mix,1,1,0,0,Unknown,Female,Intact
9,Cat,White/Black,Domestic Shorthair,Mix,3,1,0,0,Offsite,Female,Sterilized


In [29]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()

In [30]:
# Преобразовываем категориальные значения в числа с помощью LabelEncoder

label_animal_type = label_enc.fit_transform(data['animal_type'])
label_color = label_enc.fit_transform(data['color'])
label_outcome_subtype = label_enc.fit_transform(data['outcome_subtype_'])
# label_outcome_type = label_enc.fit_transform(data['outcome_type'])
label_breed_ = label_enc.fit_transform(data['breed_'])
label_breed_purity = label_enc.fit_transform(data['breed_purity'])
label_sex_on_outcome = label_enc.fit_transform(data['sex_on_outcome_'])
label_reproduction = label_enc.fit_transform(data['reproduction_'])

In [31]:
data.shape

(78237, 11)

In [32]:
# Cоздаем новые столбцы с числами вместо категорий

data['l_animal_type'] = pd.DataFrame(label_animal_type)
data['l_color'] = pd.DataFrame(label_color)
data['l_outcome_subtype'] = pd.DataFrame(label_outcome_subtype)
# data['l_outcome_type'] = pd.DataFrame(label_outcome_type)
data['l_breed'] = pd.DataFrame(label_breed_)
data['l_breed_purity'] = pd.DataFrame(label_breed_purity)
data['l_sex_on_outcome'] = pd.DataFrame(label_sex_on_outcome)
data['l_reproduction'] = pd.DataFrame(label_reproduction)

In [33]:
len(pd.DataFrame(label_animal_type))

78237

In [34]:
# Произошла странная вещь: пропали 18 последних значений из новых столбцов, хотя длины датафреймов были равны до объединения.
# В обоих случаях было 78237. Затем последние 18 стали None. Как такое могло произойти?

In [35]:
# Удалим столбцы, которые мы заменили на числовые

data.drop(['animal_type', 'color', 'breed_', 'breed_purity', 'outcome_subtype_', 'sex_on_outcome_', 'reproduction_'], axis=1, inplace=True)
data = data[:-18]
data

,age_on_outcome_in_months,adoption,transfer,other_outcome_type,l_animal_type,l_color,l_outcome_subtype,l_breed,l_breed_purity,l_sex_on_outcome,l_reproduction
0,0,0,1,0,1.0,321.0,12.0,859.0,0.0,1.0,0.0
1,12,0,1,0,2.0,474.0,12.0,236.0,0.0,0.0,1.0
2,12,1,0,0,2.0,119.0,19.0,1457.0,1.0,1.0,1.0
3,108,0,1,0,2.0,463.0,12.0,1334.0,0.0,1.0,1.0
4,5,0,0,1,4.0,122.0,14.0,235.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
78232,1,1,0,0,2.0,182.0,6.0,1008.0,0.0,1.0,1.0
78233,12,0,1,0,2.0,182.0,6.0,1008.0,0.0,0.0,1.0
78234,84,0,1,0,2.0,119.0,19.0,1284.0,0.0,1.0,1.0
78235,72,0,0,1,2.0,339.0,19.0,1796.0,1.0,0.0,1.0


In [36]:
# Отделяем атрибуты от классов

output_y = data.iloc[:,1:4]
input_x = data.iloc[:, [0, 4, 5, 6, 7, 8, 9, 10]]

In [37]:
# from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
targets=(output_y.iloc[:,:]==1).idxmax(1)
Y=le.fit_transform(targets)
print(targets.value_counts())

adoption              33098
transfer              23497
other_outcome_type    21624
dtype: int64


In [38]:
# Разделим данные на 70% тренировочных и на 30% тестовых

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    input_x,
    Y,
    test_size = 0.3
)

In [39]:
from sklearn.ensemble import RandomForestClassifier

# Создаем модель деревьев решений и выбираем 100 деревьев в качестве параметра

model=RandomForestClassifier(n_estimators=100)

# Обучаем модель

model.fit(X_train, y_train)

RandomForestClassifier()

In [40]:
# Делаем classification report

from sklearn.metrics import classification_report

model_pred = model.predict(X_test)

print(classification_report(y_test, model_pred))

model.score(X_test, y_test)

              precision    recall  f1-score   support

           0       0.54      0.61      0.57      9924
           1       0.41      0.40      0.41      6454
           2       0.49      0.41      0.45      7088

    accuracy                           0.49     23466
   macro avg       0.48      0.47      0.47     23466
weighted avg       0.49      0.49      0.49     23466



0.4906673485042189

In [41]:
# Полагаю, низкое значение получилось из-за истории с 18 пропавшими значениями, из-за которых все могло съехать.
# Но объяснения этому, к сожалению, я так и не нашел.